# Configurations

A configuration is essentially a set of (coordinate) frames, where each frame can represent a shape, joint, inertia, etc. This tutorial introduces to basics of creating & loading configurations, the joint and frame state, computing features, and handling the view window.

In [1]:
from robotic import ry
import numpy as np
import time
print(ry.compiled())

compile time: Sep 23 2023 12:43:33


## Adding frames to a configuration

The starting point is to create a `Configuration`:

In [2]:
C = ry.Config()
C.view()

0

This shows an empty configuration. Tip: Make the view window appear "Always On Top" (right click on the window bar)

A configuration is essentially a tree (or forrest) of frames. You usually add models from files, but let's do it manually here.

In [3]:
C.clear()
f = C.addFrame(name='first')
f.setShape(type=ry.ST.marker, size=[.4])
f.setPosition([0.,0.,.5])
f.setQuaternion([1., .3, .0, .0]) #is normalized internally
print('frame name:', f.name, 'pos:', f.getPosition(), 'quat:', f.getQuaternion())
C.view()

frame name: first pos: [0.  0.  0.5] quat: [0.95782629 0.28734789 0.         0.        ]


0

Let's add a second frame, but with first as parent and with a hinge joint!

In [4]:
f = C.addFrame(name='second', parent='first')
f.setJoint(ry.JT.hingeX)
f.setShape(type=ry.ST.marker, size=[.4])
f.setColor([1,0,0])
print('frame name:', f.name, 'pos:', f.getPosition(), 'quat:', f.getQuaternion())
C.view()

frame name: second pos: [0.  0.  0.5] quat: [-0.95782629 -0.28734789 -0.         -0.        ]


0

Since we now have a configuration with a joint, we can articulate it:

In [5]:
q = C.getJointState()
q[0] = q[0] + .1
C.setJointState(q)
print('joint state:', q)
C.view()

joint state: [0.1]


0

Other examples to add:

In [ ]:
C.addFrame('ball', 'second') .setShape(ry.ST.sphere, [.1]) .setColor([1.,.5,.0]) .setRelativePosition([-.3,.0,.2])
C.addFrame('box', 'second') .setShape(ry.ST.ssBox, [.3,.2,.1,.02]) .setColor([.5,1.,.0]) .setRelativePosition([.0,.0,.2])
C.addFrame('capsule', 'second') .setShape(ry.ST.capsule, [.3, .05]) .setColor([.0,1.,.5]) .setRelativePosition([.3,.0,.2])
for t in range(100):
    C.setJointState([np.cos(.1*t)])
    C.view()
    time.sleep(.1)

## Loading existing configurations

In [ ]:
C.clear()
C.addFile(ry.raiPath('panda/panda.g'))
C.view()

Let's add a second panda, but prefix all frame names, and move it to the side

In [ ]:
C.addFile(ry.raiPath('panda/panda.g'), 'r_')
base_r = C.getFrame('r_panda_base')
base_r.setPosition([.0, .5, .0])
C.view()

We can get the joint state of the full configuration:

In [ ]:
print(C.getJointState())
print('joints:', C.getJointNames())
print('frames:', C.getFrameNames())

Let's animate:

In [ ]:
q0 = C.getJointState()
for t in range(20):
    q = q0 + .1*np.random.randn(q0.shape[0])
    C.setJointState(q)
    C.view()
    time.sleep(.2)

## Features: computing geometric properties
For every frame we can query its pose:

In [ ]:
f = C.getFrame('r_gripper')
print('gripper pos:', f.getPosition())
print('gripper quat:', f.getQuaternion())
print('gripper rot:', f.getRotationMatrix())

The above provides basic forward kinematics: After `setJointState` you can query the pose of any configuration frame. However, there is a more general way to query *features*, i.e. properties of the configuration in a differentiable manner. You might not use this often; but it is important to understand as these differentiable features are the foundation of how optimization problems are formulated, which you'll need a lot.

Here are some example features to evaluate:

In [ ]:
[y,J] = C.eval(ry.FS.position, ['gripper'])
print('position of gripper:', y, '\nJacobian:', J)

In [ ]:
# negative(!) distance between two convex shapes (or origin of marker)
C.eval(ry.FS.negDistance, ['panda_coll7', 'r_panda_coll7'])

In [ ]:
# the x-axis of the given frame in world coordinates
C.eval(ry.FS.vectorX, ['gripper'])

## Joint and Frame State

A configuration is a tree of n frames. Every frame has a pose (position & quaternion), which is represented as a 7D vector (x,y,z, qw,qx,qy,qz). The frame state is the $n\times 7$ matrix, where the i-th row is the pose of the i-th frame.

A configuration also defines joints, which means that the relative transfromation from a parent to a child frame is parameterized by degrees-of-freedoms (DOFs). If the configuration has in total n DOFs, the joint state is a n-dimensional vector.

Setting the joint state implies computing all relative transformations, and then forward chaining all transformations to compute all frame poses. So setting the joint state also sets the frame state.
     
Setting the frame state allows you to set frame poses that are inconsistent/impossible w.r.t. the joints! Setting the frame state implies computing all relative transformations from the frame poses, and then assigning the joint state to the *projection* onto the actual DOFs

In [ ]:
C.setJointState(q0)
C.view()

The *frame state* is a $n\times 7$ matrix, which contains for all of $n$ frames the 7D pose. A pose is stored as [p_x, p_y, p_z, q_w, q_x, q_y, q_z], with position p and quaternion q.

In [ ]:
X0 = C.getFrameState()
print('frame state: ', X0)

Let's do a very questionable thing: adding .1 to all numbers in the frame matrix!

In [ ]:
X = X0 + .1
C.setFrameState(X)
C.view()

That totally broke the original design of the robot! Setting global frame states overwrites the relative transformations between frames.

(Also, the rows of X have non-normalized quaternions! These are normalized when setting the frame state.)

Let's reset:

In [ ]:
C.setFrameState(X0)
C.view()

## View interaction and releasing objects

You can close and re-open the view window

In [ ]:
C.view_close()

In [ ]:
# things are still there
C.view(pause=False, message='this is a message')

For user interaction it is often useful to wait for a keypress (making `view` a blocking call):

keypressed = C.view(True, 'press some key!')
print('pressed key:', keypressed, chr(keypressed))

Get a screenshot:

In [ ]:
img = C.view_getScreenshot()
print(type(img), img.shape)

And release everything, including closing the view

In [ ]:
del C